集合的本质是许多唯一对象的聚集。因此，集合可以用于去重

In [2]:
l = ['spam', 'spam', 'eggs', 'spam']
list(set(l))

['spam', 'eggs']

集合中的元素必须是可散列的，set 类型本身是不可散列的，但是
frozenset 可以。因此可以创建一个包含不同 frozenset 的 set。
除了保证唯一性，集合还实现了很多基础的中缀运算符。给定两个集合
a 和 b，a | b 返回的是它们的合集，a & b 得到的是交集，而 a - b
得到的是差集。合理地利用这些操作，不仅能够让代码的行数变少，还
能减少 Python 程序的运行时间。这样做同时也是为了让代码更易读，从
而更容易判断程序的正确性，因为利用这些运算符可以省去不必要的循
环和逻辑操作。
例如，我们有一个电子邮件地址的集合（haystack），还要维护一个
较小的电子邮件地址集合（needles），然后求出 needles 中有多少地
址同时也出现在了 heystack 里。借助集合操作，我们只需要一行代码
就可以了

In [ ]:
found = len(needles & haystack)

### 集合字面量
除空集之外，集合的字面量——{1}、{1, 2}，等等——看起来跟它的
数学形式一模一样。如果是空集，那么必须写成 set() 的形式
像 {1, 2, 3} 这种字面量句法相比于构造方法（set([1, 2, 3])）要
更快且更易读。后者的速度要慢一些，因为 Python 必须先从 set 这个
名字来查询构造方法，然后新建一个列表，最后再把这个列表传入到构
造方法里。但是如果是像 {1, 2, 3} 这样的字面量，Python 会利用一
个专门的叫作 BUILD_SET 的字节码来创建集合

用 dis.dis（反汇编函数）来看看两个方法的字节码的不同

In [3]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              3 BUILD_SET                1
              6 RETURN_VALUE


In [4]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              3 LOAD_CONST               0 (1)
              6 BUILD_LIST               1
              9 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             12 RETURN_VALUE


➊ 检查 {1} 字面量背后的字节码。  
➋ 特殊的字节码 BUILD_SET 几乎完成了所有的工作。  
➌ set([1]) 的字节码。  
➍ 3 种不同的操作代替了上面的  
BUILD_SET：LOAD_NAME、BUILD_LIST 和 CALL_FUNCTION。  

### dict和set的背后

想要理解 Python 里字典和集合类型的长处和弱点， 它们背后的散列表是
绕不开的一环。   
这一节将会回答以下几个问题。  
Python 里的 dict 和 set 的效率有多高？  
为什么它们是无序的？  
为什么并不是所有的 Python 对象都可以当作 dict 的键或 set 里的
元素？  
为什么 dict 的键和 set 元素的顺序是跟据它们被添加的次序而定
的， 以及为什么在映射对象的生命周期中， 这个顺序并不是一成不
变的？  
为什么不应该在迭代循环 dict 或是 set 的同时往里添加元素？
为了让你有动力研究散列表， 下面先来看一个关于 dict 和 set 效率的
实验， 实验对象里大概有上百万个元素， 而实验结果可能会出乎你的意
料

最快的时间来自“集合交集花费时间”这
一列， 这一列的结果是示例 3-15 中利用集合 & 操作的代码的效果。 不
出所料的是， 最糟糕的表现来自“列表花费时间”这一列。 由于列表的背
后没有散列表来支持 in 运算符， 每次搜索都需要扫描一次完整的列
表， 导致所需的时间跟据 haystack 的大小呈线性增长

散列表其实是一个稀疏数组（总是有空白元素的数组称为稀疏数组） 。
在一般的数据结构教材中， 散列表里的单元通常叫作表元（bucket） 。
在 dict 的散列表当中， 每个键值对都占用一个表元， 每个表元都有两
个部分， 一个是对键的引用， 另一个是对值的引用。 因为所有表元的大
小一致， 所以可以通过偏移量来读取某个表元。          
因为 Python 会设法保证大概还有三分之一的表元是空的， 所以在快要达
到这个阈值的时候， 原有的散列表会被复制到一个更大的空间里面。如果要把一个对象放入散列表， 那么首先要计算这个元素键的散列值。
Python 中可以用 hash() 方法来做这件事情， 接下来会介绍这一点。

### 散列值和相等性
内置的 hash() 方法可以用于所有的内置类型对象。 如果是自定义
对象调用 hash() 的话， 实际上运行的是自定义的 __hash__。 如
果两个对象在比较的时候是相等的， 那它们的散列值必须相等， 否
则散列表就不能正常运行了。 例如， 如果 1 == 1.0 为真， 那么
hash(1) == hash(1.0) 也必须为真， 但其实这两个数字（整型
和浮点） 的内部结构是完全不一样的。

In [1]:
1 == 1.0

True

In [2]:
hash(1) == hash(1.0)

True

为了让散列值能够胜任散列表索引这一角色， 它们必须在索引空间
中尽量分散开来。 这意味着在最理想的状况下， 越是相似但不相等
的对象， 它们散列值的差别应该越大。 示例 3-16 是一段代码输
出， 这段代码被用来比较散列值的二进制表达的不同。 注意其中 1
和 1.0 的散列值是相同的， 而 1.0001、 1.0002 和 1.0003 的散列值则
非常不同。

### 为什么不直接匹配检验 search_key == found_key ？
为了获取 my_dict[search_key] 背后的值， Python 首先会调用
hash(search_key) 来计算 search_key 的散列值， 把这个值最低
的几位数字当作偏移量， 在散列表里查找表元（具体取几位， 得看
当前散列表的大小） 。 若找到的表元是空的， 则抛出 KeyError 异
常。 若不是空的， 则表元里会有一对 found_key:found_value。
这时候 Python 会检验 search_key == found_key 是否为真， 如
果它们相等的话， 就会返回 found_value。

如果 search_key 和 found_key 不匹配的话， 这种情况称为散列
冲突。 发生这种情况是因为， 散列表所做的其实是把随机的元素映
射到只有几位的数字上， 而散列表本身的索引又只依赖于这个数字
的一部分。 为了解决散列冲突， 算法会在散列值中另外再取几位，
然后用特殊的方法处理一下， 把新得到的数字再当作索引来寻找表
元。 若这次找到的表元是空的， 则同样抛出 KeyError； 若非
空， 或者键匹配， 则返回这个值； 或者又发现了散列冲突， 则重复
以上的步骤。

由于字典使用了散列表， 而散列表又必须是稀疏的， 这导致它在空
间上的效率低下。 举例而言， 如果你需要存放数量巨大的记录， 那
么放在由元组或是具名元组构成的列表中会是比较好的选择； 最好不要根据 JSON 的风格， 用由字典组成的列表来存放这些记录。 用
元组取代字典就能节省空间的原因有两个： 其一是避免了散列表所
耗费的空间， 其二是无需把记录中字段的名字在每个元素里都存一
遍。

### 记住我们现在讨论的是空间优化。 如果你手头有几百万个对象， 而你的机器有几个 GB 的内存， 那么空间的优化工作可以等到真正需要的时候再开始计划， 因为优化往往是可维护性的对立面。